In [ ]:
# syft absolute
import syft as sy

In [ ]:
high_client = sy.login(
    url="http://localhost", port=8080, email="info@openmined.org", password="changethis"
)

In [ ]:
bae980ecdebda81ca370b90115bb2e00e704d512bf310349ea16ba098f10c46a

In [ ]:
str(high_client.credentials)

In [ ]:
dockerfile_list = high_client.images.get_all()

In [ ]:
base_worker_image = next(
    (
        image
        for image in dockerfile_list
        if image.is_prebuilt and "syft-backend" in str(image.config)
    ),
    None,
)
base_worker_image

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery

""".strip()
worker_dockerfile

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=worker_dockerfile)

In [ ]:
# docker_config = sy.DockerWorkerConfig(dockerfile=worker_dockerfile)
assert docker_config.dockerfile == worker_dockerfile

In [ ]:
submit_result = high_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
worker_image = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == worker_dockerfile
    ),
    None,
)
worker_image

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
result = high_client.api.services.image_registry.add("k3d-registry.localhost:5800")

In [ ]:
image_registry_list = high_client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
local_registry = "k3d-registry.localhost:5800"

In [ ]:
local_registry = image_registry_list[0]
local_registry

In [ ]:
docker_build_result = high_client.api.services.worker_image.build(
    image_uid=worker_image.id,
    tag=docker_tag,
    registry_uid=local_registry.id,
)
print(docker_build_result)

In [ ]:
push_result = high_client.api.services.worker_image.push(worker_image.id)

In [ ]:
result = high_client.api.services.worker_pool.launch(
    pool_name="mypool",
    image_uid=worker_image.id,
    num_workers=1,
)
result